# 包导入

In [1]:
import numpy as np
import pandas as pd

# 数据导入

In [16]:
stone  = pd.read_excel("stone.xlsx")
print("护石:")
print(stone.head())

skill = pd.read_excel("skill.xlsx")
print("技能:")
print(skill.head())

护石:
   等级      一技能  Lv    二技能  Lv.1     孔  value
0   7  火属性攻击强化   1     不屈   1.0  21.0      0
1   6     昏厥耐性   2    击晕术   1.0  11.0      0
2   5      植生学   1  毒属性强化   2.0  11.0      0
3   6     风压耐性   1    击晕术   1.0  11.0      0
4   7       逆袭   2     利刃   1.0   1.0      1
技能:
  LookUpName  MaxLevel
0         攻击         7
1        挑战者         5
2         无伤         3
3         怨恨         5
4       死里逃生         3


# 数据预处理

In [18]:
# 处理缺失值
stone.fillna({"二技能":"无","Lv.1":0},inplace=True)

#处理技能,将技能用数字替换
skillArray = skill["LookUpName"].values
stone['一技能'].replace(skillArray,list(range(1,113)),inplace=True)
stone['二技能'].replace(skillArray,list(range(1,113)),inplace=True)
stone.head(20)

,等级,一技能,Lv,二技能,Lv.1,孔,value
0,7,13,1,92,1.0,21.0,0
1,6,76,2,41,1.0,11.0,0
2,5,79,1,18,2.0,11.0,0
3,6,62,1,41,1.0,11.0,0
4,7,106,2,23,1.0,1.0,1
5,4,40,3,16,1.0,0.0,0
6,6,67,2,69,1.0,21.0,0
7,5,70,1,13,2.0,11.0,0
8,5,18,2,53,1.0,1.0,0
9,7,76,1,62,1.0,21.0,0


# 模型一尝试

# 模型二尝试

# 模型三尝试